## Proyecto final procesamiento de lenguaje natural

**Integrantes:**

* Laura del Pilar Torres Toro - Código: 617202012
* Gabriel Cruz Parra - Código: 617202013

**Planteamiento del Problema:**

* Determinar la variación de opinión de x personajes políticos en los últimos 10 años respecto a un tema determinado.

**Motivación:**

* Determinar si en la clase política colombiana existe una tendencia a contradecir sus convicciones.

**Metodología:**

* Adquisición de los datos 
* Pre procesamiento
* Clasificación
* Determinación de variación

### Desarrollo del proyecto

* Obtenemos por entrada de teclado un conjunto de parámetros deseados por el usuario:

In [44]:
import snscrape.modules.twitter as sntwitter
from datetime import datetime
from datetime import timedelta
import pandas as pd
import re
import six
import ipywidgets as widgets
from google.cloud import translate_v2 as translate
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from flair.models import TextClassifier
from flair.data import Sentence

import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

vader = SentimentIntensityAnalyzer()

In [22]:
listadoPoliticos = pd.read_excel('./sampleFiles/listadoPoliticos.xlsx')

listadoTemas = pd.read_excel('./sampleFiles/listadoPoliticos.xlsx', sheet_name='Temas')
listadoTemas

,Tema
0,Aborto
1,La inmigración ilegal
2,La violencia de pareja y de género
3,Legalizacion de drogas
4,Subsidios
5,Pena de muerte
6,Legalización de armas
7,Extincion de dominio
8,Reforma tributaria
9,Reforma a la Salud


In [23]:
opcionesPoliticos= []
for index, row in listadoPoliticos.iterrows():
    opcionesPoliticos.append((row["Politico"]+" - "+row["Cargo"], row["Cuenta"]))

opcionesTemas = []
for index, row in listadoTemas.iterrows():
    opcionesTemas.append(row["Tema"])

opcionesTemas

['Aborto',
 'La inmigración ilegal',
 'La violencia de pareja y de género',
 'Legalizacion de drogas',
 'Subsidios',
 'Pena de muerte',
 'Legalización de armas',
 'Extincion de dominio',
 'Reforma tributaria',
 'Reforma a la Salud',
 'Reforma Pensional',
 'Educación gratuita',
 'Matrimonio igualitario',
 'Infraestructura']

In [59]:
    
dropdown_politicos = widgets.Dropdown(options =opcionesPoliticos, description="Político: ")
dropdown_temas = widgets.Dropdown(options =opcionesTemas, description="Temas: ")
display(dropdown_politicos)
display(dropdown_temas)
slider_vigencias = widgets.IntSlider(description="Años de retro búsqueda: ")
text_vigencias = widgets.IntText()
display(slider_vigencias, text_vigencias)
widgets.jslink((slider_vigencias, 'value'), (text_vigencias, 'value'))

#vigencias = int(input('Digite el número de años para retro búsqueda: '))
#tema = input('Digite el tema deseado: ')

   

Dropdown(description='Político: ', options=(('Juan Manuel Santos - Ex Presidente', '@JuanManSantos'), ('Álvaro…

Dropdown(description='Temas: ', options=('Aborto', 'La inmigración ilegal', 'La violencia de pareja y de géner…

IntSlider(value=0, description='Años de retro búsqueda: ')

IntText(value=0)

Link(source=(IntSlider(value=0, description='Años de retro búsqueda: '), 'value'), target=(IntText(value=0), '…

In [61]:

politico = dropdown_politicos.value
tema = dropdown_temas.value
now = datetime.now()
retro_search = now - timedelta(days = (365*text_vigencias.value))
hoy = now.strftime("%Y-%m-%d")
pasado = retro_search.strftime("%Y-%m-%d")
print('desde: ', pasado, 'hasta', hoy)
tweet_list = []
maxTweets = 100  
# petrogustavo
for i,tweet in enumerate(sntwitter.TwitterSearchScraper(f'{tema} since:{pasado} until:{hoy} from:{politico}').get_items()) :
    if i > maxTweets :
        break
    print(tweet)
    print(tweet.date)
    print(tweet.content)
    print("*"*100)
    print()
        
    tweet_list.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.user.displayname, tweet])
    
# Creating a dataframe from the tweets list above 
tweets_df = pd.DataFrame(tweet_list, columns=['Fecha', 'tweet_id', 'contenido', 'usuario', 'usuario_mostrar', 'enlace'])
tweets_df.head() 

desde:  2001-06-04 hasta 2021-05-30
https://twitter.com/AlvaroUribeVel/status/1396969678170435586
2021-05-24 23:21:40+00:00
Cierto: 
Al Pte Duque le montaron saboteo y violencia desde su elección;

Pero: 
Aumento d pobreza por pandemia y reforma tributaria generaron gran inconformidad social;

Necesidad:
Políticas sociales y recursos pagados por ls más pudientes;
Autoridad frente violencia y bloqueos
****************************************************************************************************

https://twitter.com/AlvaroUribeVel/status/1396154864393232396
2021-05-22 17:23:53+00:00
La reforma tributaria retirada fracasó primero en la opinión pública, después en el Congreso.

La nueva, que pagarían los más pudientes para financiar la política social necesita que el Gbno la legitime primero ante la opinión pública.
****************************************************************************************************

https://twitter.com/AlvaroUribeVel/status/1396130727738159105
2021-0

https://twitter.com/AlvaroUribeVel/status/538291035462725632
2014-11-28 11:19:18+00:00
En minutos otros 13 comentarios sobre Reforma Tributaria
****************************************************************************************************



,Fecha,tweet_id,contenido,usuario,usuario_mostrar,enlace
0,2021-05-24 23:21:40+00:00,1396969678170435586,Cierto: \nAl Pte Duque le montaron saboteo y v...,AlvaroUribeVel,Álvaro Uribe Vélez,https://twitter.com/AlvaroUribeVel/status/1396...
1,2021-05-22 17:23:53+00:00,1396154864393232396,La reforma tributaria retirada fracasó primero...,AlvaroUribeVel,Álvaro Uribe Vélez,https://twitter.com/AlvaroUribeVel/status/1396...
2,2021-05-22 15:47:59+00:00,1396130727738159105,Si la reforma tributaria la pagarían solamente...,AlvaroUribeVel,Álvaro Uribe Vélez,https://twitter.com/AlvaroUribeVel/status/1396...
3,2021-05-13 12:58:06+00:00,1392826487145914373,"La violencia que ha aparecido en estos días, n...",AlvaroUribeVel,Álvaro Uribe Vélez,https://twitter.com/AlvaroUribeVel/status/1392...
4,2021-05-01 12:34:33+00:00,1388471905708089345,"Grupos violentos organizados, \n\ncon terroris...",AlvaroUribeVel,Álvaro Uribe Vélez,https://twitter.com/AlvaroUribeVel/status/1388...


In [37]:
# Definición de función de pre-procesamiento
stopwords_sp = stopwords.words('spanish')

def preProcesado(texto):
    pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
    pattern2 = '?P<url>https?://[^\s]+'
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",texto)]
    for link in links:
        texto = texto.replace(link,"")
    
    hashtags = [interaction for interaction in texto.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        texto = texto.replace(hashtag, "")
        
    mentions = [interaction for interaction in texto.split() if interaction.startswith("@")]
    for mention in mentions:
        texto = texto.replace(mention, "")
    
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = texto.split() # Tokenizar
    texto = [palabra for palabra in texto if palabra not in stopwords_sp]
    texto = [palabra for palabra in texto if len(palabra)>=2]
    texto = " ".join(texto)
    return texto

In [64]:
tweets_df['pre_procesado'] = tweets_df['contenido'].apply(lambda x: preProcesado(x))
tweets_df.head()

,Fecha,tweet_id,contenido,usuario,usuario_mostrar,enlace,pre_procesado
0,2021-05-24 23:21:40+00:00,1396969678170435586,Cierto: \nAl Pte Duque le montaron saboteo y v...,AlvaroUribeVel,Álvaro Uribe Vélez,https://twitter.com/AlvaroUribeVel/status/1396...,cierto pte duque montaron saboteo violencia el...
1,2021-05-22 17:23:53+00:00,1396154864393232396,La reforma tributaria retirada fracasó primero...,AlvaroUribeVel,Álvaro Uribe Vélez,https://twitter.com/AlvaroUribeVel/status/1396...,reforma tributaria retirada fracasó primero op...
2,2021-05-22 15:47:59+00:00,1396130727738159105,Si la reforma tributaria la pagarían solamente...,AlvaroUribeVel,Álvaro Uribe Vélez,https://twitter.com/AlvaroUribeVel/status/1396...,si reforma tributaria pagarían solamente pudie...
3,2021-05-13 12:58:06+00:00,1392826487145914373,"La violencia que ha aparecido en estos días, n...",AlvaroUribeVel,Álvaro Uribe Vélez,https://twitter.com/AlvaroUribeVel/status/1392...,violencia aparecido días sido espontánea preme...
4,2021-05-01 12:34:33+00:00,1388471905708089345,"Grupos violentos organizados, \n\ncon terroris...",AlvaroUribeVel,Álvaro Uribe Vélez,https://twitter.com/AlvaroUribeVel/status/1388...,grupos violentos organizados terrorismo premed...


In [65]:
translate_client = translate.Client()

def traducir(texto, lenguaje):
    if isinstance(texto, six.binary_type):
        texto = texto.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    resultado = translate_client.translate(texto, target_language=lenguaje)
    return resultado['translatedText']

In [66]:
lenguaje = 'en'
tweets_df['traducido'] = tweets_df['pre_procesado'].apply(lambda x: traducir(x,lenguaje))
tweets_df.head()

,Fecha,tweet_id,contenido,usuario,usuario_mostrar,enlace,pre_procesado,traducido
0,2021-05-24 23:21:40+00:00,1396969678170435586,Cierto: \nAl Pte Duque le montaron saboteo y v...,AlvaroUribeVel,Álvaro Uribe Vélez,https://twitter.com/AlvaroUribeVel/status/1396...,cierto pte duque montaron saboteo violencia el...,certain pte duke mounted sabotage violence ele...
1,2021-05-22 17:23:53+00:00,1396154864393232396,La reforma tributaria retirada fracasó primero...,AlvaroUribeVel,Álvaro Uribe Vélez,https://twitter.com/AlvaroUribeVel/status/1396...,reforma tributaria retirada fracasó primero op...,tax reform withdrawn failed first public opini...
2,2021-05-22 15:47:59+00:00,1396130727738159105,Si la reforma tributaria la pagarían solamente...,AlvaroUribeVel,Álvaro Uribe Vélez,https://twitter.com/AlvaroUribeVel/status/1396...,si reforma tributaria pagarían solamente pudie...,if tax reform would pay only the wealthy expan...
3,2021-05-13 12:58:06+00:00,1392826487145914373,"La violencia que ha aparecido en estos días, n...",AlvaroUribeVel,Álvaro Uribe Vélez,https://twitter.com/AlvaroUribeVel/status/1392...,violencia aparecido días sido espontánea preme...,violence appeared days been spontaneous premed...
4,2021-05-01 12:34:33+00:00,1388471905708089345,"Grupos violentos organizados, \n\ncon terroris...",AlvaroUribeVel,Álvaro Uribe Vélez,https://twitter.com/AlvaroUribeVel/status/1388...,grupos violentos organizados terrorismo premed...,organized violent groups premeditated terroris...


In [67]:
# realización de análisis de sentimientos para cada tweet flair

classifier = TextClassifier.load('sentiment')

def sent_flair(texto):
    sentence = Sentence(texto)
    classifier.predict(sentence)
    result = sentence.labels[0]
    label = result.value
    score = result.score
    if label == 'POSITIVE':
        return score
    if label == 'NEGATIVE':
        return -1 * score
    return score

2021-05-30 22:50:31,847 loading file C:\Users\gabri\.flair\models\sentiment-en-mix-distillbert_4.pt


In [69]:
tweets_df['textblob_sentiment_en'] = tweets_df['traducido'].apply(lambda x: TextBlob(x).sentiment.polarity)
tweets_df['vader_sentiment_en'] = tweets_df['traducido'].apply(lambda x: vader.polarity_scores(x)['compound'])
tweets_df['flair_sentiment_en'] = tweets_df['traducido'].apply(lambda x: sent_flair(x))
tweets_df

,Fecha,tweet_id,contenido,usuario,usuario_mostrar,enlace,pre_procesado,traducido,textblob_sentiment_en,vader_sentiment_en,flair_sentiment_en
0,2021-05-24 23:21:40+00:00,1396969678170435586,Cierto: \nAl Pte Duque le montaron saboteo y v...,AlvaroUribeVel,Álvaro Uribe Vélez,https://twitter.com/AlvaroUribeVel/status/1396...,cierto pte duque montaron saboteo violencia el...,certain pte duke mounted sabotage violence ele...,0.316190,-0.7964,-0.940768
1,2021-05-22 17:23:53+00:00,1396154864393232396,La reforma tributaria retirada fracasó primero...,AlvaroUribeVel,Álvaro Uribe Vélez,https://twitter.com/AlvaroUribeVel/status/1396...,reforma tributaria retirada fracasó primero op...,tax reform withdrawn failed first public opini...,0.074411,-0.2960,-0.985570
2,2021-05-22 15:47:59+00:00,1396130727738159105,Si la reforma tributaria la pagarían solamente...,AlvaroUribeVel,Álvaro Uribe Vélez,https://twitter.com/AlvaroUribeVel/status/1396...,si reforma tributaria pagarían solamente pudie...,if tax reform would pay only the wealthy expan...,0.011111,0.0516,-0.981856
3,2021-05-13 12:58:06+00:00,1392826487145914373,"La violencia que ha aparecido en estos días, n...",AlvaroUribeVel,Álvaro Uribe Vélez,https://twitter.com/AlvaroUribeVel/status/1392...,violencia aparecido días sido espontánea preme...,violence appeared days been spontaneous premed...,0.411111,-0.8591,-0.991423
4,2021-05-01 12:34:33+00:00,1388471905708089345,"Grupos violentos organizados, \n\ncon terroris...",AlvaroUribeVel,Álvaro Uribe Vélez,https://twitter.com/AlvaroUribeVel/status/1388...,grupos violentos organizados terrorismo premed...,organized violent groups premeditated terroris...,-0.400000,-0.8689,-0.991381
...,...,...,...,...,...,...,...,...,...,...,...
96,2014-12-01 11:43:32+00:00,539384297854361600,Con la reforma tributaria Santos golpeará a la...,AlvaroUribeVel,Álvaro Uribe Vélez,https://twitter.com/AlvaroUribeVel/status/5393...,reforma tributaria santos golpeará inversión p...,Santos tax reform will hit investment damage r...,0.000000,-0.8442,-0.566604
97,2014-11-29 13:32:59+00:00,538687067694563328,Reforma Tributaria afecta negativamente el 70 ...,AlvaroUribeVel,Álvaro Uribe Vélez,https://twitter.com/AlvaroUribeVel/status/5386...,reforma tributaria afecta negativamente ciento...,tax reform negatively affects one hundred empl...,-0.300000,0.0000,-0.998459
98,2014-11-28 19:49:09+00:00,538419344758550530,@elespectador Reforma Tributaria afecta genera...,AlvaroUribeVel,Álvaro Uribe Vélez,https://twitter.com/AlvaroUribeVel/status/5384...,reforma tributaria afecta generación empleo uribe,Tax reform affects job creation in Uribe,0.000000,0.2732,0.559491
99,2014-11-28 19:41:38+00:00,538417452066304000,Vía @noticierodelafm Reforma tributaría va en ...,AlvaroUribeVel,Álvaro Uribe Vélez,https://twitter.com/AlvaroUribeVel/status/5384...,vía reforma tributaría va pobres álvaro uribe,via tax reform goes poor Álvaro Uribe,-0.400000,-0.4767,-0.989045


In [70]:
tweets_df['Año'] = tweets_df['Fecha'].dt.year
tweets_df.head()

,Fecha,tweet_id,contenido,usuario,usuario_mostrar,enlace,pre_procesado,traducido,textblob_sentiment_en,vader_sentiment_en,flair_sentiment_en,Año
0,2021-05-24 23:21:40+00:00,1396969678170435586,Cierto: \nAl Pte Duque le montaron saboteo y v...,AlvaroUribeVel,Álvaro Uribe Vélez,https://twitter.com/AlvaroUribeVel/status/1396...,cierto pte duque montaron saboteo violencia el...,certain pte duke mounted sabotage violence ele...,0.316190,-0.7964,-0.940768,2021
1,2021-05-22 17:23:53+00:00,1396154864393232396,La reforma tributaria retirada fracasó primero...,AlvaroUribeVel,Álvaro Uribe Vélez,https://twitter.com/AlvaroUribeVel/status/1396...,reforma tributaria retirada fracasó primero op...,tax reform withdrawn failed first public opini...,0.074411,-0.2960,-0.985570,2021
2,2021-05-22 15:47:59+00:00,1396130727738159105,Si la reforma tributaria la pagarían solamente...,AlvaroUribeVel,Álvaro Uribe Vélez,https://twitter.com/AlvaroUribeVel/status/1396...,si reforma tributaria pagarían solamente pudie...,if tax reform would pay only the wealthy expan...,0.011111,0.0516,-0.981856,2021
3,2021-05-13 12:58:06+00:00,1392826487145914373,"La violencia que ha aparecido en estos días, n...",AlvaroUribeVel,Álvaro Uribe Vélez,https://twitter.com/AlvaroUribeVel/status/1392...,violencia aparecido días sido espontánea preme...,violence appeared days been spontaneous premed...,0.411111,-0.8591,-0.991423,2021
4,2021-05-01 12:34:33+00:00,1388471905708089345,"Grupos violentos organizados, \n\ncon terroris...",AlvaroUribeVel,Álvaro Uribe Vélez,https://twitter.com/AlvaroUribeVel/status/1388...,grupos violentos organizados terrorismo premed...,organized violent groups premeditated terroris...,-0.400000,-0.8689,-0.991381,2021


In [71]:
#Filtros flair en inglés
def get_positivos(frame, columna):
    filtro = frame[frame[columna]>0.0]
    return filtro.count()

def get_negativos(frame, columna):
    filtro = frame[frame[columna]<0.0]
    return filtro.count()

def get_neutros(frame, columna):
    filtro = frame[frame[columna]==0.0]
    return filtro.count()


# conteos flair inglés
conteoPositivosFlairEn = get_positivos(tweets_df[['flair_sentiment_en']],'flair_sentiment_en')
conteoNegativosFlairEn = get_negativos(tweets_df[['flair_sentiment_en']],'flair_sentiment_en')
conteoNeutrosFlairEn = get_neutros(tweets_df[['flair_sentiment_en']],'flair_sentiment_en')

# conteos vader inglés
conteoPositivosVaderEn = get_positivos(tweets_df[['vader_sentiment_en']],'vader_sentiment_en')
conteoNegativosVaderEn = get_negativos(tweets_df[['vader_sentiment_en']],'vader_sentiment_en')
conteoNeutrosVaderEn = get_neutros(tweets_df[['vader_sentiment_en']],'vader_sentiment_en')
# conteos textblob inglés
conteoPositivosTextblobEn = get_positivos(tweets_df[['textblob_sentiment_en']],'textblob_sentiment_en')
conteoNegativosTextblobEn = get_negativos(tweets_df[['textblob_sentiment_en']],'textblob_sentiment_en')
conteoNeutrosTextblobEn = get_neutros(tweets_df[['textblob_sentiment_en']],'textblob_sentiment_en')


conteosData = pd.concat([conteoPositivosFlairEn, conteoNegativosFlairEn, conteoNeutrosFlairEn,
                        conteoPositivosVaderEn, conteoNegativosVaderEn, conteoNeutrosVaderEn,
                        conteoPositivosTextblobEn, conteoNegativosTextblobEn, conteoNeutrosTextblobEn], axis= 0)

listaValores = ['Positivo flair (en)', 'Negativo flair (en)', 'Neutro flair (en)',
               'Positivo vader (en)', 'Negativo vader (en)', 'Neutro vader (en)',
               'Positivo textblob (en)', 'Negativo textblob (en)', 'Neutro textblob (en)']
conteosData =pd.DataFrame(conteosData).reset_index()

conteosData.columns = ['Atributo','conteo']


for i in (range(len(conteosData))):
    conteosData.loc[i:i,'Atributo'] = listaValores[i]

conteosData

,Atributo,conteo
0,Positivo flair (en),41
1,Negativo flair (en),60
2,Neutro flair (en),0
3,Positivo vader (en),28
4,Negativo vader (en),36
5,Neutro vader (en),37
6,Positivo textblob (en),37
7,Negativo textblob (en),29
8,Neutro textblob (en),35
